In [2]:
# Instalamos el modelo a utilizar
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=40c944ac3da0c3684c2c621fec63fbf879e2e0bf745ca7eeb3e4828bb8f774f5
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [3]:
#Comenzamos importando las librerias a utilizar
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from lightfm import LightFM

In [6]:
#importamos el dataframe a trabajar
df = pd.read_csv("/content/ratings_Electronics.csv")
df

,user,prod,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1.365811e+09
1,A2CX7LUOHB2NDG,0321732944,5.0,1.341101e+09
2,A2NWSAGRHCP8N5,0439886341,1.0,1.367194e+09
3,A2WNBOD3WNDNKT,0439886341,3.0,1.374451e+09
4,A1GI0U4ZRJA8WN,0439886341,1.0,1.334707e+09
...,...,...,...,...
154909,A1G7VCQUH95FWE,B000067S60,1.0,1.378080e+09
154910,A28Y1YD0QK88M5,B000067S60,5.0,1.395014e+09
154911,A1YK9ACFOAMRLK,B000067S60,3.0,1.379203e+09
154912,AZLDKR28KT3FB,B000067S60,4.0,1.398298e+09


#Hagamos un pequeño EDA


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154914 entries, 0 to 154913
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       154914 non-null  object 
 1   prod       154913 non-null  object 
 2   rating     154913 non-null  float64
 3   timestamp  154913 non-null  float64
dtypes: float64(2), object(2)
memory usage: 4.7+ MB


In [8]:
df.describe()

,rating,timestamp
count,154913.000000,1.549130e+05
mean,4.024801,1.203319e+09
std,1.361033,1.546796e+08
min,1.000000,9.127296e+08
25%,4.000000,1.036541e+09
50%,5.000000,1.232842e+09
75%,5.000000,1.357517e+09
max,5.000000,1.405987e+09


In [9]:
#veamos si hay nulos
df.isna().sum()

user         0
prod         1
rating       1
timestamp    1
dtype: int64

In [10]:
#Como se trata de una sola fila y parece ser un error de carga de datos, directamente la vamos a descartar
df = df.drop(154913	, axis = 0)

In [11]:
df.user.nunique()

136638

#Preprocesamiento de los datos



In [12]:
#Acomodamos la columna timestamp para trabajar con las fechas, creando una función

def get_ts(ts):
  date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
  return date


In [13]:
#Aplicamos la función a todo el dataset
df['timestamp'] = df['timestamp'].apply(get_ts)

In [14]:
#Veamos desde que valores tenemos las fechas
print(df['timestamp'].min())
print(df['timestamp'].max())

1998-12-04
2014-07-22


In [15]:
#Vemos el tipo de dato
df.timestamp.dtype

dtype('O')

In [16]:
#Cambiamos el tipo de dato para trabajar mejor
df["timestamp"] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')

In [17]:
#Volvemos a confirmar que se haya hecho el cambio
df.timestamp.dtype

dtype('<M8[ns]')

In [18]:
#Vemos la cantidad de compras hechas por año
plot_df = df.copy()
plot_df['year'] = df.timestamp.dt.year
plot_df = plot_df.groupby('year', as_index=False).count()[['year', 'user']]
plot_df.columns = ['year', 'products_count']
plot_df

,year,products_count
0,1998,4
1,1999,1212
2,2000,9214
3,2001,14753
4,2002,15830
5,2003,8111
6,2004,4159
7,2005,4160
8,2006,4811
9,2007,7683


In [71]:
#Dividimos el set en train y test
#El set de train desde el año 2010 hasta el 2013 pero en el mes de Noviembre para dejar diciembre de ese año en test
train = df[np.logical_and(df['timestamp'] > datetime(year=2010,month=12,day=31), df['timestamp'] < datetime(year=2013,month=11,day=1))]

In [72]:
test = df[np.logical_and(df['timestamp']>= datetime(year=2013,month=12,day=1),df['timestamp']<=datetime(year=2013,month=12,day=31))]

In [73]:
#veamos las cantidades de ambos sets
print(train.shape)
print(test.shape)

(44455, 4)
(2437, 4)


In [74]:
#Veamos cuantos cold start tenemos en el set de train
#COMPLETAR
test_user = set(test['user'])
train_user = set(train['user'])
userIds_not_in_train = test_user - train_user

display(userIds_not_in_train)

display(len(userIds_not_in_train))

{'A2LOQXC3XDSBI8',
 'AASWBLOFKE7JI',
 'A343ENF730UN71',
 'A1N5M2F7T2PE8Q',
 'AVLGNAULPWSY1',
 'A2M5TSHIMFO5NN',
 'A3EML7WV7A5QZW',
 'A4QXAGLC2Y8B8',
 'A3VPL62PWGWJ4I',
 'A10I2XBWWYM3LL',
 'APTP07QR8Z4OW',
 'AGNMQ4QQK7LUY',
 'A1KCOUPV4SZMA3',
 'A3U4AIFD3JJZSH',
 'A25KCPBVN5SWQP',
 'A29MX2PGKYHE8E',
 'A1UDAK9UX1D1YT',
 'A18D3YC9NTGU1W',
 'A195ND08U5BAXT',
 'ABG742CKZS6L5',
 'A2IUXC55PAOY06',
 'A2IY383UW7991O',
 'AC4K062IBCZIE',
 'A32B9EFHAYFS97',
 'A32RZM38ACY0Y0',
 'A621VS6VITAI4',
 'A2WFEMS2MF85AM',
 'A14D0CYVZWVS88',
 'A318XAP8UZZWVU',
 'A34HEHB3Z0YHO3',
 'A28M1DLNFZSX1B',
 'A1HNY10HIAV6YP',
 'A2C3WOBWKHAPBU',
 'A2BUIT9Y3HXF3B',
 'A22GKJVZ05V1LA',
 'A3MLT7V3UE6RI8',
 'A21MDR14ZX3OXG',
 'A1W02ERLTAEMDH',
 'A36QJUI4G5BXQ5',
 'ANAXC5HPVM7OX',
 'A3QSTIHCISMTL3',
 'A2XKZ8LHP4RUVC',
 'A3QTGBLB20GPDR',
 'AU84TW6TV0XZE',
 'A1M7SKAUQ2QK0K',
 'A3BJLE99R4ZWVF',
 'A1KYJ2CV9IJUYD',
 'A301ZMQ2FPX1AG',
 'A1VJDYVBTZQNKU',
 'A22EOFSMSLH1XF',
 'A2OZ96KRM10ST1',
 'A3NL4I41MB9E91',
 'A3MRZZO6CXJONO',
 'A

2278

In [75]:
#Generamos la matriz de interacciones
matrix_columns =['prod','rating','user']
interactions = train[matrix_columns]
interactions.head()

,prod,rating,user
0,0132793040,5.0,AKM1MP6P0OYPR
1,0321732944,5.0,A2CX7LUOHB2NDG
2,0439886341,1.0,A2NWSAGRHCP8N5
3,0439886341,3.0,A2WNBOD3WNDNKT
4,0439886341,1.0,A1GI0U4ZRJA8WN


In [76]:
#utilizamos pivot para armar la matriz
interactions_matrix = train.pivot(index = 'user', columns ='prod', values = 'rating')

In [77]:
#Reemplazamos los NaN con 0
interactions_matrix = interactions_matrix.fillna(0)

In [78]:
interactions_matrix.shape

(42376, 3856)

In [79]:
#Transformamos la matrix a una csr matrix
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

In [80]:
#Generamos un diccionario con los usuarios como indice
users_id = list(interactions_matrix.index)
user_dict = {}
counter = 0
for i in users_id:
  user_dict[i] = counter
  counter += 1

In [81]:
user_dict

{'A00625243BI8W1SSZNLMD': 0,
 'A00766851QZZUBOVF4JFT': 1,
 'A00995931BE16NG4F52QC': 2,
 'A0103849GBVWICKXD4T6': 3,
 'A014623426J5CM7M12MBW': 4,
 'A02459613IS599WTHNWWS': 5,
 'A0293130VTX2ZXA70JQS': 6,
 'A030530627MK66BD8V4LN': 7,
 'A03279253KKB83JP34CU': 8,
 'A0402564TCEO67AUZFJO': 9,
 'A04256643L1ZMT75HVD2P': 10,
 'A0478499AHWR7GZCCWR1': 11,
 'A05039072BTS8XOPJKBXB': 12,
 'A055760637EV7Z72T33OR': 13,
 'A0586418108XHQA8T1IQO': 14,
 'A06374853B2OXBKB6H1YT': 15,
 'A0641581307AKT5MAOU0Q': 16,
 'A0677968MUIZV1URPJAN': 17,
 'A0686610O9FM6Y3JAN2J': 18,
 'A07324873GQMRSBW9534Y': 19,
 'A076219533YHEV2LJO988': 20,
 'A07730732C0MK8X9APP6C': 21,
 'A0783501NVHNISYJ7CY8': 22,
 'A07865442XT0VEAVPSGX1': 23,
 'A0815906MFVDTIFHGRQR': 24,
 'A0821988FXKFYX53V4QG': 25,
 'A085830328D6R4Z74JTJ': 26,
 'A085969913PPYQX9MGCJ4': 27,
 'A08871831JFLDQ8W8RJ8Y': 28,
 'A09271781JOQU3Y4IT8QB': 29,
 'A092907613PUH3WG01CIR': 30,
 'A09600262CLBSRBGU2VTY': 31,
 'A09778421NQ9P4ZZ7HFA': 32,
 'A1004RESD21CDE': 33,
 'A1008Z4

#Modelo de recomendación


In [82]:
#Instanciamos el modelo
model = LightFM(no_components = 10,random_state = 42,learning_rate = 0.03)

In [83]:
#Entremaos el modelo
%%time
model = model.fit(interactions_matrix_csr, epochs=100)

CPU times: user 5.13 s, sys: 13 ms, total: 5.14 s
Wall time: 6.39 s


In [84]:
#Veamos como se generan las predicciones
user_x = user_dict['A12ZI4T9QB81BA']
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)
#ACA GENERAMOS PREDICCIONES
preds = model.predict(user_ids=user_x, item_ids = item_ids)

In [85]:
#Vemos las recomendaciones
scores = pd.Series(preds)
scores.index = interactions_matrix.columns
recomms_5 = list(pd.Series(scores.sort_values(ascending=False).index))[:10] #obtenemos las primeras 10
print(recomms_5)

['B00004ZCJE', 'B00004T8R2', 'B00001P4ZH', 'B00001WRSJ', 'B00005T3G0', 'B00005N6KG', '0972683275', 'B000067RC4', 'B000067RT6', 'B000065BPB']


In [86]:
 user_5_purchase = train[(train.user == 'A12ZI4T9QB81BA')]['prod']

In [87]:
user_5_purchase

47961    B00004TDSC
Name: prod, dtype: object

In [88]:
#Filtramos los productos que compró ese Userr
recomms_5 = [rec for rec in recomms_5 if rec not in user_5_purchase]

In [89]:
#Creamos un df con los productos más comprados
df_popularity = train.groupby('prod',as_index = False).agg({'user':'nunique'}).sort_values(by = 'user',ascending = False)
df_popularity.columns = ['prod','popularity']
print(df_popularity.head(50))


            prod  popularity
2144  B00004ZCJE        1273
1501  B00004T8R2         994
1127  B00001P4ZH         978
3170  B00005T3G0         945
1176  B00001WRSJ         933
2907  B00005N6KG         880
25    0972683275         758
3764  B000067RC4         659
3836  B000067RT6         658
3623  B000065BPB         553
3728  B000067O5G         539
1416  B00004SABB         518
3579  B0000645RH         478
3415  B000062VUO         409
2618  B00005ATMB         408
2879  B00005ML7Q         402
43    1400532655         400
1734  B00004WCGF         387
1928  B00004Z5M1         387
2823  B00005LEN4         379
3621  B000065BP9         369
2108  B00004ZC8Y         362
1542  B00004THCZ         357
44    140053271X         354
2620  B00005ATMI         340
3842  B000067RVL         321
2755  B00005K47X         279
1078  B00000K2YR         251
3199  B00005T3N3         250
1125  B00001P4XH         244
2561  B00005A1K1         239
1259  B00002EQCW         236
1605  B00004TWM6         233
1739  B00004WC

In [90]:
#Ordenamos el dataframe para quedarnos con los 20 más populares
df_popularity = df_popularity.sort_values(by = 'popularity',ascending = False).head(20)
df_popularity

,prod,popularity
2144,B00004ZCJE,1273
1501,B00004T8R2,994
1127,B00001P4ZH,978
3170,B00005T3G0,945
1176,B00001WRSJ,933
2907,B00005N6KG,880
25,0972683275,758
3764,B000067RC4,659
3836,B000067RT6,658
3623,B000065BPB,553


In [91]:
df_popularity['prod']

2144    B00004ZCJE
1501    B00004T8R2
1127    B00001P4ZH
3170    B00005T3G0
1176    B00001WRSJ
2907    B00005N6KG
25      0972683275
3764    B000067RC4
3836    B000067RT6
3623    B000065BPB
3728    B000067O5G
1416    B00004SABB
3579    B0000645RH
3415    B000062VUO
2618    B00005ATMB
2879    B00005ML7Q
43      1400532655
1734    B00004WCGF
1928    B00004Z5M1
2823    B00005LEN4
Name: prod, dtype: object

In [92]:
from tqdm import tqdm

In [93]:
#definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'cold': []
}

#obtenemos  usuarios y cantidad de productos
n_users,n_prods = interactions_matrix.shape
id_items = np.arange(n_prods)
#por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(test.user.unique()):


    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    #COMPLETAR:
    if user in list(interactions_matrix.index):


      userx = user_dict[user] #buscamos el indice del usuario en la matriz (transformamos id a indice)


      # Generar las predicciones para el usuario x
      preds = model.predict(user_ids=userx, item_ids = id_items)



      # Nos quedamos con las 20 mejores recomendaciones
      scores = pd.Series(preds)
      scores.index = interactions_matrix.columns
      recomms = list(pd.Series(scores.sort_values(ascending=False).index))[:20]

      # Listado de productos comporados por el usuario
      prod_purchased = train[train['user'] == userx]['prod'].unique()


      # Filtramos el contenido ya visto y nos quedamos con las 20 recomendaciones
      recomms = [rec for rec in scores if rec not in prod_purchased ][:20]


      # Guardamos las recomendaciones en el diccionario
      recomms_dict['user_id'].append(user)
      recomms_dict['recomms'].append(scores)
      recomms_dict['cold'].append(False)

    # En este else trataremos a los usuarios que no están en la matriz (cold start)
    else:
      recomms_dict['user_id'].append(user)
      # Les recomendamos contenido popular
      recomms_dict['recomms'].append(list(df_popularity['prod']))
      recomms_dict['cold'].append(True)



100%|██████████| 2365/2365 [00:19<00:00, 120.32it/s]


In [94]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,cold
0,A253JJFXQNPCOJ,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
1,A2TKKYL3GKFS2M,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
2,A2DUI0YVE1DZ48,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
3,A39J1PTX1KAL66,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
4,A3EI9TX2A4MUSZ,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
...,...,...,...
2360,A3J4SC9M4JQEX6,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
2361,A162BKW9GLUTPH,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
2362,A3T2Q4B3O5J067,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True
2363,AC1WSO610CU6,"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G...",True


In [95]:
recomms_df.cold.value_counts()

cold
True     2278
False      87
Name: count, dtype: int64

In [96]:
#Generamos un listado de productos ideal que son los productos que ya compraron para compararlo con las recomendaciones
ideal_recomms = test.sort_values(by =["user","prod"], ascending=False).groupby(["user"], as_index=False).agg({"prod": "unique"}).head()
ideal_recomms.head()

,user,prod
0,A02850745KLGY02ZALLI,[B000067RC4]
1,A0957734330PLRZT884UU,[B00004XOM3]
2,A1027EV8A9PV1O,[B00001P4ZH]
3,A102SDBNUQ6W72,[B00001P4ZH]
4,A104KDXA8NQOUV,[B00005ATMB]


In [97]:
#Unimos el df ideal con el df real
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user", right_on="user_id")[["user", "prod", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,A02850745KLGY02ZALLI,[B000067RC4],"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G..."
1,A0957734330PLRZT884UU,[B00004XOM3],"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G..."
2,A1027EV8A9PV1O,[B00001P4ZH],prod 0132793040 0.799055 0321732944 0.79...
3,A102SDBNUQ6W72,[B00001P4ZH],"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G..."
4,A104KDXA8NQOUV,[B00005ATMB],"[B00004ZCJE, B00004T8R2, B00001P4ZH, B00005T3G..."


In [98]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [99]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.03
